# Random Ensemble Orchestration - With get STATE


## RL test

Defining the Simulation enviroment

Running the simulation

In [ ]:
from stable_baselines3 import PPO
from managers.enviroment_manager import SimulationEnv
#setup_config()

MODEL_PATH = "simulation_policy.zip"


# Create the environment
env = SimulationEnv("ppo")

if os.path.exists(MODEL_PATH):
    print("Loading existing model...")
    model = PPO.load(MODEL_PATH, env=env)

# Train the agent
model = PPO("MlpPolicy", env, verbose=1, n_steps=1028)
model.learn(total_timesteps=20)

# Save the trained model
model.save("simulation_policy")


Contents of c:\Documents\Aalto Semester 2\ResearchCloud\RL_simulation\initial_sim_config.yaml have been dumped into c:\Documents\Aalto Semester 2\ResearchCloud\RL_simulation\sim_config.yaml.


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\Filippo\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning:

You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.

c:\Users\Filippo\anaconda3\Lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning:

You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 1028`, after every 16 untruncated mini-batches, there will be a truncated mini-batch of size 4
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=1028 and n_envs=1)

2025-02-08 19:09:52,297:INFO -- Action: Adding model at index 4


Model 'ResNet50' successfully added to the target YAML.
STATE:  {'ensemble_state': {'total_energy_consumption': 0.000645155062259547, 'ensemble_size': 2}, 'model_states': {'MobileNetV2': {'accuracy': 0.6666666666666666, 'confidence': 0.5069998701413473, 'avg_response_time': 0.027011485943924243, 'max_response_time': 0.039131649029821435, 'contribution': 0.5069998701413473}, 'ensemble': {'accuracy': 0.6666666666666666, 'confidence': 0.5664711892604828, 'avg_response_time': 0.16096994169966047, 'max_response_time': 0.4203466046038169, 'contribution': 0.6259425083796183}, 'ResNet50': {'accuracy': 1.0, 'confidence': 0.9008195400238037, 'avg_response_time': 0.4138814927424761, 'max_response_time': 0.4138814927424761, 'contribution': 0.9008195400238037}, 'padding_model_4': {'accuracy': 0.0, 'confidence': 0.0, 'avg_response_time': 0.0, 'max_response_time': 0.0, 'contribution': 0.0}, 'padding_model_5': {'accuracy': 0.0, 'confidence': 0.0, 'avg_response_time': 0.0, 'max_response_time': 0.0, 'co

2025-02-08 19:09:52,627:INFO -- Action: Removing model at index 1
2025-02-08 19:09:52,740:INFO -- Action: Adding model at index 1


...

Model 'ResNet50' successfully added to the target YAML.
STATE:  {'ensemble_state': {'total_energy_consumption': 0.0011837247471048551, 'ensemble_size': 3}, 'model_states': {'MobileNetV2': {'accuracy': 0.7, 'confidence': 0.5555651307106018, 'avg_response_time': 0.10299900824196415, 'max_response_time': 0.12189439080346721, 'contribution': 0.6226602017879486}, 'ensemble': {'accuracy': 0.9, 'confidence': 0.7765687753756841, 'avg_response_time': 1.176069291315194, 'max_response_time': 1.2096933568490844, 'contribution': 0.9344332933425903}, 'ResNet50': {'accuracy': 0.8, 'confidence': 0.689080822467804, 'avg_response_time': 0.4037563866762775, 'max_response_time': 0.4271851751435289, 'contribution': 0.7349778652191162}, 'EfficientNetV2S': {'accuracy': 0.9, 'confidence': 0.82376047372818, 'avg_response_time': 1.166169015248952, 'max_response_time': 1.1981784149541193, 'contribution': 0.906129789352417}, 'NASNetLarge': {'accuracy': 1.0, 'confidence': 0.8785170912742615, 'avg_response_time': 3

2025-02-08 19:12:45,119:INFO -- Action: Removing model at index 4
2025-02-08 19:12:45,313:INFO -- Action: Adding model at index 2


STATE:  {'ensemble_state': {'total_energy_consumption': 0.0013332306055035144, 'ensemble_size': 4}, 'model_states': {'MobileNetV2': {'accuracy': 0.7, 'confidence': 0.5302370309829711, 'avg_response_time': 0.10377812360915994, 'max_response_time': 0.12189439080346721, 'contribution': 0.597332102060318}, 'ensemble': {'accuracy': 0.9, 'confidence': 0.7601736774047216, 'avg_response_time': 1.1790831210132424, 'max_response_time': 1.2096933568490844, 'contribution': 0.9249052047729492}, 'ResNet50': {'accuracy': 0.8, 'confidence': 0.6795527338981628, 'avg_response_time': 0.40565816380366415, 'max_response_time': 0.4271851751435289, 'contribution': 0.7254497766494751}, 'EfficientNetV2S': {'accuracy': 0.9, 'confidence': 0.8094313681125641, 'avg_response_time': 1.1690671477723966, 'max_response_time': 1.1981784149541193, 'contribution': 0.8918006837368011}, 'NASNetLarge': {'accuracy': 1.0, 'confidence': 0.8785170912742615, 'avg_response_time': 3.8143150357465627, 'max_response_time': 3.84251730

2025-02-08 19:12:45,530:INFO -- Action: Removing model at index 0
2025-02-08 19:12:45,681:INFO -- Action: Adding model at index 2


Model 'DenseNet121' successfully removed from the target YAML.
STATE:  {'ensemble_state': {'total_energy_consumption': 0.0005728913335178048, 'ensemble_size': 2}, 'model_states': {'MobileNetV2': {'accuracy': 0.8, 'confidence': 0.6251321315765381, 'avg_response_time': 0.10004665407115065, 'max_response_time': 0.12189439080346721, 'contribution': 0.6251321315765381}, 'ensemble': {'accuracy': 1.0, 'confidence': 0.8543476561705271, 'avg_response_time': 1.1717925436943701, 'max_response_time': 1.1991901071148545, 'contribution': 0.9334855914115906}, 'ResNet50': {'accuracy': 0.8, 'confidence': 0.6795527338981628, 'avg_response_time': 0.40565816380366415, 'max_response_time': 0.4271851751435289, 'contribution': 0.7254497766494751}, 'EfficientNetV2S': {'accuracy': 1.0, 'confidence': 0.9030818819999695, 'avg_response_time': 1.1616042936631827, 'max_response_time': 1.1903052676253203, 'contribution': 0.9030818819999695}, 'NASNetLarge': {'accuracy': 1.0, 'confidence': 0.8785170912742615, 'avg_res